In [ ]:
# 🚀 Auto-setup: installs deps + configures CFBD access
%run ./_auto_setup.py


# 12 · Update Training Data

Use this notebook as a living runbook for refreshing `model_pack/updated_training_data.csv` whenever new CFBD data arrives. The workflow is now automated by `model_pack/scripts/build_training_dataset.py` and the `refresh-training` subcommand in `project_management/data_workflows.py`.

**Pipeline overview**

1. **Migrate raw starter-pack games** to the 86-feature schema via `StarterPackDataMigrator`.
2. **Extend** `updated_training_data.csv` with the migrated rows (deduped on `game_key`).
3. **Retrain** models (optional) so notebooks/agents immediately consume the latest weights.

> 🔑 Requires `CFBD_API_KEY` plus the optional dependencies from `requirements-optional.txt` if you intend to regenerate features from scratch.


In [ ]:
import shlex
import subprocess

command = [
    "python",
    "../project_management/data_workflows.py",
    "refresh-training",
    "--max-week",
    "13",
    "--train-models",
]

print("Running:\n", " ".join(shlex.quote(part) for part in command))
subprocess.run(command, check=False)



## CLI reference

```
python project_management/data_workflows.py refresh-training \
  --max-week 13 \              # optional filter for in-progress seasons
  --skip-migration \           # if you already have a migrated CSV
  --migrated-file path.csv \   # custom migrated CSV when skipping migration
  --skip-extend \              # preview output without touching updated_training_data.csv
  --train-models \             # retrain FastAI neural model after extension
  --epochs 400 \
  --learning-rate 5e-4
```

The command internally calls `model_pack/scripts/build_training_dataset.py`, so you can run that script directly if you prefer to stay inside the model_pack directory.
